In [92]:
#Importando librerías necesarias 

import psycopg2
from psycopg2.extras import execute_values
import requests
import pandas as pd
from datetime import datetime
import datetime as dt

In [93]:
#Estableciendo la conexión con la base de datos 

with open("C:/Users/Carolina/Desktop/password.txt",'r') as file:
    pwd= file.read()
host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
db='data-engineer-database'
user='carolina_cargnelutti_coderhouse'

try: 
    conn = psycopg2.connect(
        host=host,
        database=db,
        password=pwd,
        port=5439,
        user=user      
  )
    cursor = conn.cursor()
    print('Se realizó la conexión correctamente ')

except Exception as e:
    print('No fue posible realizar la conexión')

Se realizó la conexión correctamente 


In [94]:
#Creando la tabla en la base de datos

query="CREATE TABLE IF NOT EXISTS carolina_cargnelutti_coderhouse.weather(id INT primary key not null, location VARCHAR NOT NULL,region VARCHAR,temperature INTEGER,condition VARCHAR,wind INT,wind_direction VARCHAR,humidity INT,feels_like INTEGER ,visibility INTEGER,updated DATETIME, date_transfered DATETIME)"
cursor.execute(query)
conn.commit()

In [95]:
#Comprobando que la tabla esta vacía. En el caso de tener registros, estos se eliminan:

cursor.execute('SELECT * FROM carolina_cargnelutti_coderhouse.weather')
weather_table=cursor.fetchall()
if len(weather_table)>0:
    cursor.execute('TRUNCATE TABLE carolina_cargnelutti_coderhouse.weather')
    print('Se eliminaron correctamente todos los registros de la tabla')
else:
    print('La tabla no tiene registros')

Se eliminaron correctamente todos los registros de la tabla


In [96]:
#Extrayendo datos de la API

counter=1

def request_api(localidad):
    url='http://api.weatherapi.com/v1/forecast.json?key=9bc72106866d42c28da193618240701 &q='+str(localidad)+'&days=1&aqi=yes&alerts=no'
    response=requests.get(url).json()
    return response


date_transfered=dt.datetime.today()

data={'id':[],'location':[],'region':[],'temperature':[],'condition':[], 'wind':[], 'wind_direction':[], 'humidity':[],
     'feels_like': [], 'visibility':[],'updated':[], 'date_transfered': []}

locations= ['Buenos Aires','Catamarca','Córdoba','Cataratas del Iguazú','Comodoro Rivadavia','Concordia','El Calafate','Ezeiza','Formosa','Jujuy','Mar del Plata','Mendoza','Neuquen','Paraná','Posadas','Puerto Madryn',
    'Reconquista','Salta','San Carlos de Bariloche', 'Rio Gallegos', 'Termas de Rio Hondo', 'Ushuaia']

for location in locations:
    response=request_api(location)
    location= response['location']['name']
    region= response['location']['region']
    temperature =response['current']['temp_c']
    feels_like= response['current']['feelslike_c']
    condition = response['current']['condition']['text']
    wind = response['current']['wind_kph']
    wind_direction=response['current']['wind_dir']
    humidity=response['current']['humidity']
    visibility=response['current']['vis_km']
    updated=response['current']['last_updated']

    data['id'].append(counter)
    data['location'].append(location)
    data['region'].append(region)
    data['temperature'].append(temperature)
    data['condition'].append(condition)
    data['wind'].append(wind)
    data['wind_direction'].append(wind_direction)
    data['humidity'].append(humidity)
    data['feels_like'].append(feels_like)
    data['visibility'].append(visibility)
    data['updated'].append(updated)
    data['date_transfered'].append(date_transfered)
    
    counter = counter + 1
df=pd.DataFrame(data)

In [97]:
#Limpieza del dataframe:
def limpieza(df):
    df.dropna()
    df.drop_duplicates(subset='location')
    return df

def modif_datos(df):
    df['temperature']=df['temperature'].astype('int')
    df['feels_like']=df['feels_like'].astype('int')
    df['wind']=df['wind'].astype('int')
    df['humidity']=df['humidity'].astype('int')
    df['visibility']=df['visibility'].astype('int')
    df['updated'] =pd.to_datetime(df['updated'] )
    return df

df=limpieza(df)
df=modif_datos(df)
df.head(5)

,id,location,region,temperature,condition,wind,wind_direction,humidity,feels_like,visibility,updated,date_transfered
0,1,Buenos Aires,Distrito Federal,15,Partly cloudy,15,NE,82,14,10,2024-05-04 10:30:00,2024-05-04 15:41:13.006575
1,2,Catamarca,Catamarca,15,Sunny,3,NNE,58,15,10,2024-05-04 10:30:00,2024-05-04 15:41:13.006575
2,3,Córdoba,Cordoba,13,Sunny,16,NNE,67,11,10,2024-05-04 10:30:00,2024-05-04 15:41:13.006575
3,4,Cataratas Del Iguazu,Misiones,23,Moderate or heavy rain with thunder,3,NNE,94,25,10,2024-05-04 10:30:00,2024-05-04 15:41:13.006575
4,5,Comodoro Rivadavia,Distrito Federal,7,Sunny,19,WNW,61,2,10,2024-05-04 10:30:00,2024-05-04 15:41:13.006575


In [98]:
#Actualizando la fecha de inserción de los datos e insertándolos en la base de datos:

df.date_transfered=dt.datetime.today()
execute_values(cursor,
'''
INSERT INTO carolina_cargnelutti_coderhouse.weather (id,location,region,temperature,condition,wind,wind_direction,humidity,feels_like,visibility,updated, date_transfered)
VALUES %s
''',[tuple(x) for x in df.values],page_size=len(df))
conn.commit()

In [99]:
cursor.close()
conn.close()